<a href="https://colab.research.google.com/github/dhhieu397/Tensorflow-Certificate/blob/master/TimeSeries_Sunspot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/Sunspots.csv \
    -O /tmp/sunspots.csv

import csv
time_step = []
sunspots = []

with open('/tmp/sunspots.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader)
  for row in reader:
    sunspots.append(float(row[2]))
    time_step.append(int(row[0]))

series = np.array(sunspots)
# DO NOT CHANGE THIS CODE
# This is the normalization function
min = np.min(series)
max = np.max(series)
series -= min
series /= max
time = np.array(time_step)

split_time = 3000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 30
batch_size = 32
shuffle_buffer_size = 1000

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size,
                              shuffle_buffer=shuffle_buffer_size)
print(np.array(train_set).shape)

### Your code here
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                               strides=1, padding="causal",
                               activation="relu",
                               input_shape=[None, 1]),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.LSTM(60, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

model.compile(loss='mae', optimizer='adam', )
model.fit(train_set, batch_size=256, epochs=100)

def model_forecast(model, series, window_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(32).prefetch(1)
   forecast = model.predict(ds)
   return forecast

rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

test_val = 100 * result
score = np.ceil(17 - test_val)
if score > 5:
   score = 5

print(score)
# Lưu Model và nộp 
model.save("mymodel.h5")

--2021-09-29 17:54:04--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/Sunspots.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 173.194.210.128, 173.194.213.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70827 (69K) [application/octet-stream]
Saving to: ‘/tmp/sunspots.csv’

/tmp/sunspots.csv   100%[===================>]  69.17K  --.-KB/s    in 0.005s  

2021-09-29 17:54:05 (12.8 MB/s) - ‘/tmp/sunspots.csv’ saved [70827/70827]

()
Epoch 1/100
93/93 [==============================] - 7s 40ms/step - loss: 0.0775
Epoch 2/100
93/93 [==============================] - 4s 39ms/step - loss: 0.0527
Epoch 3/100
93/93 [==============================] - 4s 39ms/step - loss: 0.0478
Epoch 4/100
93/93 [==============================] - 4s 39ms/step - loss: 0.0458
Epoch 5/100
93/93 [==============================] - 4s 39ms/step - loss: 0.04

In [ ]:
#Cách 2: 
https://github.com/shloak26/Sunspot-Time-Series-Prediction/blob/master/Sunspot%20Activity.ipynb